# setup

In [ ]:
run_test=True
# run_test=False
epochs=(1,1,1,0)
epochs=(15,500,3,0)
cutdate='2024-12-20'
testData=''
testData='new/'
weight=(0.8,1.0)
itr=5
metrics_cnt=6
import torch,sys

if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  mydrive='/content/drive/MyDrive/volrt/'
  source_path=mydrive+"git/Crossformer"
  tables = ['volvNVDA.csv', 'volvTSLA.csv', 'volvAAPL.csv', 'volvGOOG.csv', ]
  batch_size=int(torch.cuda.get_device_properties(0).total_memory/1e9+.5)*32
  import os,sys
  if not os.path.exists(source_path):
    %cd $mydrive/git
    !git clone https://github.com/jerronl/Crossformer.git
    %cd $source_path
  else:
    %cd $source_path
    # !git reset --hard HEAD
    # !git pull origin master
  %pip install einops #scikit-learn
  sys.path.append( source_path)
else:
  tables = [ 'volvAAPL.csv']
  testData=''
  mydrive= 'E:/mydoc/git/trade/analyics/'
  batch_size=32
  sys.path.append("G:/git/Crossformer1/")
  itr=1
%cd $mydrive


In [ ]:

from cross_exp.exp_crossformer import Exp_crossformer


from utils.tools import init_args,update_args
import seaborn as sns, numpy as np,math
import matplotlib.pyplot as plt
args=init_args()
data_parser={}

def regplot(dep_var, metrics, cols, itr=None, figsize=4):
    cnt = len(dep_var) - dep_var.count("_")
    figs = min(cnt, cols)
    _, axes = plt.subplots(math.ceil(cnt / figs), figs, figsize=(figsize*figs,figsize*max(1,(cnt-1)//figs+1)))
    j = 0

    for i, name in enumerate(dep_var):
        if name != "_":
            # if itr:
            name=f'{itr}_{name}'
            axs = axes.flat[j] if figs > 1 else axes
            j = j + 1
            left, right = 999, -999
            for ii in range(len(results)):
                preds, trues, _ = results[ii]
                sns.regplot(
                    ax=axs,
                    x=trues[:, i],
                    y=preds[:, i],
                    scatter_kws={"color": f"C{ii}", "alpha": 0.3},
                    line_kws={"color": f"C{ii}", "alpha": 0.3},
                    label=labels[1][ii],
                )
                mask = ~np.isnan(trues[:, i])
                if not dep_var[i][:3] in ["dtm", "pmc"]:
                    left = min(left, max(np.min(trues[:, i][mask]), -5))
                    right = max(right, min(np.max(trues[:, i][mask]), 5))
                else:
                    left = min(left, np.min(trues[:, i][mask]))
                    right = max(right, np.max(trues[:, i][mask]))
            axs.set_title(name)
            axs.set_xlim(left=left, right=right)
            axs.legend()
    metric = []
    for ii in range(len(results)):
        _, _, m = results[ii]
        metric.append(m)

    metrics = np.append(
        metrics, np.array(metric).reshape([1, len(metric), len(m)]), axis=0
    )

    plt.show()
    return metrics
    


def plot_metric(metrics, labels, *args, **kwargs):
    a, b, c = metrics.shape
    _, axs = plt.subplots(
        nrows=math.ceil(c / 2),
        ncols=2, figsize=(16, 16), 
    )
    for i in range(c):
        ax = (
            axs[i // 2, i % 2] if c > 1 else axs
        )  # Handle the case when c=1 to avoid indexing errors
        for j in range(a):
            ax.plot(
                metrics[j, :, i], label=labels[0][j], *args, **kwargs
            )  # Plot each series in the i-th plot
        ax.set_title(labels[2][i])
        ax.legend()  # Show legend in each subplot

        # Set custom x-axis labels
        ax.set_xticks(range(b))  # Set x-tick positions for all 'b' points
        ax.set_xticklabels(labels[1])  # Set x-tick labels

    plt.tight_layout()
    plt.show()

def plot_hlc():
  global results,labels
  dep_var=data_names(data_columns(test_set),20)[0]
  labels=[[f'm{i}' for i in range(itr)],tables,["mae", "mse", "rmse", "mape", "mspe", "accr"]]
  metrics=np.empty((0,len(tables),metrics_cnt))
  n_categories=data_columns(test_set)["ycat"]
  for i in range(itr):
    setting=update_args(args,data_parser,i)
    DatasetMTS.clear()
    exp = Exp_crossformer(args)
    print(f">>>>>>>testing : {setting}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    results = []
    for table in tables:
        results.append(exp.test(setting, 'prcs', True, data_path=[table], inverse=True))
    metrics=regplot(dep_var,metrics,4,i)
        
    rows=(len(results)-1)//4+1
    _, axes = plt.subplots(rows, 4, figsize=(16, 4*rows))  # 1 row, 2 columns
    axes = axes.flatten()

    for j, result in enumerate(results):
        preds, trues, _ = result
        preds = preds[:, -n_categories:]
        trues = np.round(trues[:,0]).astype(int)
        df = pd.DataFrame(preds, columns=[f"{i}" for i in range(1, n_categories + 1)])
        df['True_Label'] = trues
        # Sum probabilities for each true category
        summed_probs = df.groupby('True_Label').mean().T
        ax = axes[j]
        for col in summed_probs.columns:
            ax.plot(summed_probs.index, summed_probs[col], label=f'Col {col}',
                    linestyle='--' if col>len(summed_probs.columns)*.67 else '-'
                    if col<len(summed_probs.columns)/3. else ':')

        # Add labels to each line
        for k,col in enumerate( summed_probs.columns):
            # icol=round(col)
            ax.text(
                x=summed_probs.index[k],  # Place at the last x-coordinate
                y=summed_probs[col].iloc[k],  # Place at the last y-coordinate
                s=col,  # Label
                fontsize=8,  # Font size for labels
                verticalalignment='center',  # Align text vertically
                horizontalalignment='left'  # Align text horizontally
            )

        # Add rug plots
        ax.hist(trues,bins=range(max(trues)),density=True)
        # ax.legend(loc='center left', bbox_to_anchor=(1.05, 0.5))
        ax.set_title(f'{tables[j]}_{i}')
        # axes[j].set_xlim(0, n_categories-1)
        axes[j].set_ylim(0.02, 0.1)
    axes[-1].legend(loc='center left', bbox_to_anchor=(1.05, 0.5))
    plt.show()
    
  print(metrics)
  plot_metric(metrics,labels)

np.set_printoptions(precision=4, suppress=True)


# train

In [ ]:
if not run_test:
    from cross_exp.exp_crossformer import Exp_crossformer
    from data.data_loader import DatasetMTS
    data_parser = {
        "vols": {
            "patience":epochs[0],
            "train_epochs":epochs[1],
            'learning_rate':0.01,
            'data_split':[0.7,0.15,0.15],
            'batch_size':batch_size*2//5,
            'e_layers':5,
            'd_model':512,
            'lradj':'type2',
            "checkpoints":"./checkpoints/",
            'root_path':mydrive,
            'data_path':tables,
        },
        }
    s=epochs[3]
    for _ in range(epochs[2]):
        data_parser["vols"]["learning_rate"]=.01
        for i in range(epochs[2]):
            for ii in range(s,itr):
                # setting record of experiments
                data_parser["vols"]['weight']=weight[0]
                setting=update_args(args,data_parser,ii)    
                DatasetMTS.clear()
                # drive.mount('/content/drive/',force_remount=True)
                !google-drive-ocamlfuse -cc
                
                exp = Exp_crossformer(args)  # set experiments
                print(f">>>>>>>start training : {setting}>>>>>>>>>>>>>>>>>>>>>>>>>>")
                exp.train(setting, "vols")

                print(f">>>>>>>testing : {setting}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
                preds, trues,_ = exp.test(setting, 'vols', True, data_path=[tables[-1]], inverse=True)
                print(preds.shape, trues.shape)

                data_parser["vols"]['weight']=weight[1]
                setting=update_args(args,data_parser,ii)
                DatasetMTS.clear()
                exp = Exp_crossformer(args)  # set experiments
                print(f">>>>>>>start training : {setting}>>>>>>>>>>>>>>>>>>>>>>>>>>")
                exp.train(setting, "prcs")
                preds, trues,_ = exp.test(setting, 'prcs', True, data_path=[tables[-1]], inverse=True)
                print(preds.shape, trues.shape)
            data_parser["vols"]["learning_rate"]/=3.
            s=0

In [ ]:
if not run_test:
    from google.colab import runtime
    runtime.unassign()

# test

In [ ]:
from data.data_loader import DatasetMTS
from data.data_def import data_columns, data_names
data_parser = {
    "vols": {
        'e_layers':5,
        'd_model':512,
        'lradj':'type2',
        'weight':weight[0],
        'root_path':mydrive,
    },
    }
test_set='vols'
dep_var=data_names(data_columns(test_set),20)[0]
labels=[[f'm{i}' for i in range(itr)],tables,["mae", "mse", "rmse", "mape", "mspe", "accr"]]
metrics=np.empty((0,len(tables),metrics_cnt))
for i in range(itr):
  setting=update_args(args,data_parser,i)    
  DatasetMTS.clear()
  exp = Exp_crossformer(args)
  print(f">>>>>>>testing : {setting}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
  results = []
  for table in tables:
      results.append(exp.test(setting, 'vols', True, data_path=[table], inverse=True))
  metrics=regplot(dep_var,metrics,4,i)

print(metrics)
plot_metric(metrics,labels)

## hlc

In [ ]:
from data.data_loader import DatasetMTS
from data.data_def import data_columns, data_names
import pandas as pd

test_set='prcs'
data_parser = {
    "vols": {
        'e_layers':5,
        'd_model':512,
        'lradj':'type2',
        'root_path':mydrive,
        'weight':0.8#weight[1],
    },
    }
plot_hlc()

## horizon


In [ ]:
from data.data_loader import DatasetMTS
from cross_exp.exp_crossformer import Exp_crossformer
test_set='vols'

labels=[[f'm{i}' for i in range(itr)] ,[f'h{h+1}' for h in range(5)],["mae", "mse", "rmse", "mape", "mspe", "accr"]]
dep_var=data_names(data_columns(test_set),20)[0]
metrics=np.empty((0,len(labels[1]),len(labels[2])))
for i in range(itr):
  results = []
  for h in range(5):
    data_parser = {
    "vols": {
        'e_layers':5,
        'd_model':512,
        'lradj':'type2',
        "query": f"floor(horizon)=={h+1}",
        'weight':weight[0],
        'root_path':mydrive,
        'data_path':tables,    
    },
    }
    setting=update_args(args,data_parser,i)
    DatasetMTS.clear()
    exp = Exp_crossformer(args)
    print(f">>>>>>>testing : {data_parser['vols']['query']} m{i}h{h+1}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    results.append(exp.test(setting, 'vols', True, inverse=True))
  metrics=regplot(dep_var,metrics,4,i)

print(metrics)
plot_metric(metrics,labels)

## cutline

In [ ]:
from cross_exp.exp_crossformer import Exp_crossformer
from data.data_loader import DatasetMTS
from data.data_def import data_columns, data_names
test_set='vols'

data_parser = {
    "vols": {
        'e_layers':5,
        'd_model':512,
        "query": f"date>'#{cutdate}'",
        'root_path':mydrive+testData,
        'weight':weight[0],    
    },
    }

dep_var=data_names(data_columns(test_set),20)[0]
labels=[[f'm{i}' for i in range(itr)],tables,["mae", "mse", "rmse", "mape", "mspe", "accr"]]
metrics=np.empty((0,len(tables),metrics_cnt))
for i in range(itr):
  setting=update_args(args,data_parser,i)
  DatasetMTS.clear()
  exp = Exp_crossformer(args)
  print(f">>>>>>>testing : {setting}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
  results = []
  for table in tables:
      results.append(exp.test(setting, 'vols', True, data_path=[table], inverse=True))
  metrics=regplot(dep_var,metrics,4,i)

print(metrics)
plot_metric(metrics,labels)

## hlc

In [ ]:
from data.data_loader import DatasetMTS
from data.data_def import data_columns, data_names
import pandas as pd

test_set='prcs'
data_parser = {
    "vols": {
        'e_layers':5,
        'd_model':512,
        'lradj':'type2',
        "query": f"date>'#{cutdate}'",
        'root_path':mydrive+testData,
        'weight':weight[1],
        },
    }
plot_hlc()

## horizon


In [ ]:
from data.data_loader import DatasetMTS
from cross_exp.exp_crossformer import Exp_crossformer
from data.data_def import data_columns, data_names
test_set='vols'
labels=[[f'm{i}' for i in range(itr)] ,[f'h{h+1}' for h in range(5)],["mae", "mse", "rmse", "mape", "mspe", "accr"]]
dep_var=data_names(data_columns(test_set),20)[0]
metrics=np.empty((0,len(labels[1]),len(labels[2])))
for i in range(itr):
  results = []
  for h in range(5):
    data_parser = {
    "vols": {ddfadsfasdfsdfdsaffdsdsfdfsasdfdfssdsasaddfasasffdasfasdfssdf232dsfasdsaa 
        'e_layers':5,
        'd_model':512,
        'lradj':'type2',
        'root_path':mydrive+testData,
        "query": f"date>'#{cutdate}' and floor(horizon)=={h+1}",
        'weight':weight[0],
        'data_path':tables,    
    },
    }
    setting=update_args(args,data_parser,i)
    DatasetMTS.clear()
    exp = Exp_crossformer(args)
    print(f">>>>>>>testing : {data_parser['vols']['query']} m{i}h{h+1}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    results.append(exp.test(setting, 'vols', True, inverse=True))
  metrics=regplot(dep_var,metrics,4,i)

print(metrics)
plot_metric(metrics,labels)

# end

In [ ]:
plt.show()
from google.colab import runtime
runtime.unassign()